Establecemos la configuración necesaria e importamos las librerías que usaremos para la extracción del dato con técnicas de web scrapping. En concreto haremos uso de Selenium para "scrapear" la información necesaria para nuestro proyecto

In [1]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

import warnings
warnings.filterwarnings('ignore')

import parsel

from selenium.webdriver.common.by import By

In [2]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [27]:
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager
#PATH = ChromeDriverManager().install()
driver = webdriver.Chrome(options = opciones)
driver.get('https://www.google.es')

La segunda fuente de información que enriquecerá nuestro database será la url "https://www.worldcyclingstats.com/es/carrera/tour-de-france". El objetivo es recolectar la tabla que contiene la información de las etapas de las que está compuesta la carrera

In [28]:
url = "https://www.worldcyclingstats.com/es/carrera/tour-de-france"

driver.get(url)

time.sleep(3)

In [29]:
tabla = driver.find_element(By.CSS_SELECTOR, 'div.container > table > tbody > tr:nth-child(1) > td.content > div.main.animated.fadeInRight > div.box.box_full > div.box-content')

tabla


<selenium.webdriver.remote.webelement.WebElement (session="7e03c2be569e6f58027ae4f1b63aa093", element="6E176FD9C66B9D51961695A15E925781_element_145")>

In [30]:
tabla.text

"Overview\nProfiles\nWithdrawals\nClassifications\n# Date *  Stage Km   Winner  Leader\n1 1 jul '23  Bilbao - Bilbao 182.0   Yates, Adam (UAD)  Yates, Adam (UAD)\n2 2 jul '23  Vitoria-Gasteiz - Donostia 208.9   Lafay, Victor (COF)  Yates, Adam (UAD)\n3 3 jul '23  Amorebieta - Bayonne 193.5   Philipsen, Jasper (ADC)  Yates, Adam (UAD)\n4 4 jul '23  Dax - Nogaro 181.8   Philipsen, Jasper (ADC)  Yates, Adam (UAD)\n5 5 jul '23  Pau - Laruns 162.7   Hindley, Jai (BOH)  Hindley, Jai (BOH)\n6 6 jul '23  Tarbes - Cauterets 144.9   Pogačar, Tadej (UAD)  Vingegaard, Jonas (TJV)\n7 7 jul '23  Mont-de-Marsan - Bordeaux 169.9   Philipsen, Jasper (ADC)  Vingegaard, Jonas (TJV)\n8 8 jul '23  Libourne - Limoges 200.7   Pedersen, Mads (LTK)  Vingegaard, Jonas (TJV)\n9 9 jul '23  Saint-Léonard-de-Noblat - Puy de.. 182.4   Woods, Michael (IPT)  Vingegaard, Jonas (TJV)\n10 11 jul '23  Vulcania - Issoire 167.2   Bilbao, Pello (TBV)  Vingegaard, Jonas (TJV)\n11 12 jul '23  Clermont-Ferrand - Moulins 179.8  

In [31]:
cuerpo = tabla.find_element(By.TAG_NAME, 'tbody')

filas = cuerpo.find_elements(By.TAG_NAME, 'tr')


In [32]:
matriz = []


for f in filas:
    
    tmp = []
    
    elems = f.find_elements(By.TAG_NAME, 'td')  # esto es una lista
    
    for e in elems:
        
        tmp.append(e.text)
        
    matriz.append(tmp)

In [33]:
df = pd.DataFrame(matriz)

df

,0,1,2,3,4,5,6,7,8,9,10
0,#,Date,*,Stage,,Km,Winner,Leader,None,None,None
1,1,1 jul '23,,,Bilbao - Bilbao,,182.0,,"Yates, Adam (UAD)",,"Yates, Adam (UAD)"
2,2,2 jul '23,,,Vitoria-Gasteiz - Donostia,,208.9,,"Lafay, Victor (COF)",,"Yates, Adam (UAD)"
3,3,3 jul '23,,,Amorebieta - Bayonne,,193.5,,"Philipsen, Jasper (ADC)",,"Yates, Adam (UAD)"
4,4,4 jul '23,,,Dax - Nogaro,,181.8,,"Philipsen, Jasper (ADC)",,"Yates, Adam (UAD)"
5,5,5 jul '23,,,Pau - Laruns,,162.7,,"Hindley, Jai (BOH)",,"Hindley, Jai (BOH)"
6,6,6 jul '23,,,Tarbes - Cauterets,,144.9,,"Pogačar, Tadej (UAD)",,"Vingegaard, Jonas (TJV)"
7,7,7 jul '23,,,Mont-de-Marsan - Bordeaux,,169.9,,"Philipsen, Jasper (ADC)",,"Vingegaard, Jonas (TJV)"
8,8,8 jul '23,,,Libourne - Limoges,,200.7,,"Pedersen, Mads (LTK)",,"Vingegaard, Jonas (TJV)"
9,9,9 jul '23,,,Saint-Léonard-de-Noblat - Puy de..,,182.4,,"Woods, Michael (IPT)",,"Vingegaard, Jonas (TJV)"


In [34]:
# voy a quedarme solo con las columnas 4,6,8,10

df = df.iloc[:,[4,6,8,10]]

df.columns = ['stage', 'length', 'winner', 'leader']

df

,stage,length,winner,leader
0,,Winner,None,None
1,Bilbao - Bilbao,182.0,"Yates, Adam (UAD)","Yates, Adam (UAD)"
2,Vitoria-Gasteiz - Donostia,208.9,"Lafay, Victor (COF)","Yates, Adam (UAD)"
3,Amorebieta - Bayonne,193.5,"Philipsen, Jasper (ADC)","Yates, Adam (UAD)"
4,Dax - Nogaro,181.8,"Philipsen, Jasper (ADC)","Yates, Adam (UAD)"
5,Pau - Laruns,162.7,"Hindley, Jai (BOH)","Hindley, Jai (BOH)"
6,Tarbes - Cauterets,144.9,"Pogačar, Tadej (UAD)","Vingegaard, Jonas (TJV)"
7,Mont-de-Marsan - Bordeaux,169.9,"Philipsen, Jasper (ADC)","Vingegaard, Jonas (TJV)"
8,Libourne - Limoges,200.7,"Pedersen, Mads (LTK)","Vingegaard, Jonas (TJV)"
9,Saint-Léonard-de-Noblat - Puy de..,182.4,"Woods, Michael (IPT)","Vingegaard, Jonas (TJV)"


In [35]:
# eliminamos la fila 0

df = df.iloc[1:,:]

df

,stage,length,winner,leader
1,Bilbao - Bilbao,182.0,"Yates, Adam (UAD)","Yates, Adam (UAD)"
2,Vitoria-Gasteiz - Donostia,208.9,"Lafay, Victor (COF)","Yates, Adam (UAD)"
3,Amorebieta - Bayonne,193.5,"Philipsen, Jasper (ADC)","Yates, Adam (UAD)"
4,Dax - Nogaro,181.8,"Philipsen, Jasper (ADC)","Yates, Adam (UAD)"
5,Pau - Laruns,162.7,"Hindley, Jai (BOH)","Hindley, Jai (BOH)"
6,Tarbes - Cauterets,144.9,"Pogačar, Tadej (UAD)","Vingegaard, Jonas (TJV)"
7,Mont-de-Marsan - Bordeaux,169.9,"Philipsen, Jasper (ADC)","Vingegaard, Jonas (TJV)"
8,Libourne - Limoges,200.7,"Pedersen, Mads (LTK)","Vingegaard, Jonas (TJV)"
9,Saint-Léonard-de-Noblat - Puy de..,182.4,"Woods, Michael (IPT)","Vingegaard, Jonas (TJV)"
10,Vulcania - Issoire,167.2,"Bilbao, Pello (TBV)","Vingegaard, Jonas (TJV)"


In [39]:
# reiniciams el índice

df.reset_index(drop=True, inplace=True)

df

,stage,length,winner,leader,stage_id
0,Bilbao - Bilbao,182.0,"Yates, Adam (UAD)","Yates, Adam (UAD)",2
1,Vitoria-Gasteiz - Donostia,208.9,"Lafay, Victor (COF)","Yates, Adam (UAD)",3
2,Amorebieta - Bayonne,193.5,"Philipsen, Jasper (ADC)","Yates, Adam (UAD)",4
3,Dax - Nogaro,181.8,"Philipsen, Jasper (ADC)","Yates, Adam (UAD)",5
4,Pau - Laruns,162.7,"Hindley, Jai (BOH)","Hindley, Jai (BOH)",6
5,Tarbes - Cauterets,144.9,"Pogačar, Tadej (UAD)","Vingegaard, Jonas (TJV)",7
6,Mont-de-Marsan - Bordeaux,169.9,"Philipsen, Jasper (ADC)","Vingegaard, Jonas (TJV)",8
7,Libourne - Limoges,200.7,"Pedersen, Mads (LTK)","Vingegaard, Jonas (TJV)",9
8,Saint-Léonard-de-Noblat - Puy de..,182.4,"Woods, Michael (IPT)","Vingegaard, Jonas (TJV)",10
9,Vulcania - Issoire,167.2,"Bilbao, Pello (TBV)","Vingegaard, Jonas (TJV)",11


In [40]:
# crear una columna stage_id con el número de etapa al principio del dataframe

df['stage_id'] = df.index + 1

df


,stage,length,winner,leader,stage_id
0,Bilbao - Bilbao,182.0,"Yates, Adam (UAD)","Yates, Adam (UAD)",1
1,Vitoria-Gasteiz - Donostia,208.9,"Lafay, Victor (COF)","Yates, Adam (UAD)",2
2,Amorebieta - Bayonne,193.5,"Philipsen, Jasper (ADC)","Yates, Adam (UAD)",3
3,Dax - Nogaro,181.8,"Philipsen, Jasper (ADC)","Yates, Adam (UAD)",4
4,Pau - Laruns,162.7,"Hindley, Jai (BOH)","Hindley, Jai (BOH)",5
5,Tarbes - Cauterets,144.9,"Pogačar, Tadej (UAD)","Vingegaard, Jonas (TJV)",6
6,Mont-de-Marsan - Bordeaux,169.9,"Philipsen, Jasper (ADC)","Vingegaard, Jonas (TJV)",7
7,Libourne - Limoges,200.7,"Pedersen, Mads (LTK)","Vingegaard, Jonas (TJV)",8
8,Saint-Léonard-de-Noblat - Puy de..,182.4,"Woods, Michael (IPT)","Vingegaard, Jonas (TJV)",9
9,Vulcania - Issoire,167.2,"Bilbao, Pello (TBV)","Vingegaard, Jonas (TJV)",10


In [41]:
# exportar el dataframe a un csv

df.to_csv('stages_clean.csv')

La tercera y última fuente de extracción de datos será la url "https://www.letour.fr/es/abandono". De la página oficial de la ronda francesa obtendremos el listado de abandonos que se han producido a lo largo de las 21 etapas

In [25]:
url = "https://www.letour.fr/es/abandono"

driver.get(url)

time.sleep(3)

In [26]:
tabla2 = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[1]/div/div')

tabla2

<selenium.webdriver.remote.webelement.WebElement (session="fa86f9502aa17f87c732a0534ed0de0a", element="B196B24C18476A3E8089FABA3B35B10A_element_60")>

In [128]:
# sacar el texto de la tabla1

tabla2.text

# dividir el texto de la tabla1 por saltos de línea

divided_data4 = tabla1.text.split('\n')

divided_data4

# separar los elementos de la tabla por espacios

divided_data5 = []

for row in divided_data4:
    divided_data5.append(row.split(' '))

divided_data5

# crear un dataframe con los datos de la tabla

df3 = pd.DataFrame(divided_data5)

df3

# quedarse solo con las columnas 0, 1, 2, 3 del dataframe

df3 = df3.iloc[:, 0:4]

df3

# eliminar la fila 0 del dataframe

df3 = df3.drop(0)

df3

,0,1,2,3
1,ETAPA,1,None,None
2,Dorsal,Corredor,Equipo,Abandono
3,131,MAS,Enric,MOVISTAR
4,ETAPA,2,None,None
5,Dorsal,Corredor,Equipo,Abandono
...,...,...,...,...
66,125,LAFAY,Victor,COFIDIS
67,ETAPA,21,None,None
68,Ningún,abandono,None,None
69,SÍGUENOS,None,None,None


In [129]:
# hacer un transpose del dataframe

df3 = df3.T

df3

,1,2,3,4,5,6,7,8,9,10,...,61,62,63,64,65,66,67,68,69,70
0,ETAPA,Dorsal,131,ETAPA,Dorsal,41,ETAPA,Ningún,ETAPA,Ningún,...,126,ETAPA,Ningún,ETAPA,Dorsal,125,ETAPA,Ningún,SÍGUENOS,Recibe
1,1,Corredor,MAS,2,Corredor,CARAPAZ,3,abandono,4,abandono,...,PEREZ,19,abandono,20,Corredor,LAFAY,21,abandono,None,información
2,None,Equipo,Enric,None,Equipo,Richard,None,None,None,None,...,Anthony,None,None,None,Equipo,Victor,None,None,None,exclusiva
3,None,Abandono,MOVISTAR,None,Abandono,EF,None,None,None,None,...,COFIDIS,None,None,None,Abandono,COFIDIS,None,None,None,del


In [130]:
# ver las columnas que solo tengan valores nulos

df3.isnull().sum()





1     2
2     0
3     0
4     2
5     0
     ..
66    0
67    2
68    2
69    3
70    0
Length: 70, dtype: int64

In [132]:
# eliminar las columnas que solo tengan valores nulos

df3 = df3.dropna(axis=1, how='all')

df3

# volver a hacer un transpose del dataframe

df3 = df3.T

df3

,0,1,2,3
1,ETAPA,1,None,None
2,Dorsal,Corredor,Equipo,Abandono
3,131,MAS,Enric,MOVISTAR
4,ETAPA,2,None,None
5,Dorsal,Corredor,Equipo,Abandono
...,...,...,...,...
66,125,LAFAY,Victor,COFIDIS
67,ETAPA,21,None,None
68,Ningún,abandono,None,None
69,SÍGUENOS,None,None,None


In [133]:
# exportar el dataframe a un csv

df3.to_csv('withdrawals.csv')

In [149]:
# importar el csv withdrawals como dataframe

df4 = pd.read_csv('withdrawals.csv')

df4

,etapa,corredor,equipo
1,1,MAS Enric,MOVISTAR
2,2,CARAPAZ Richard,EF
3,3,NaN,NaN
4,4,NaN,NaN
5,5,GUARNIERI Jacopo,LOTTO
6,5,SANCHEZ Luis Leon,NaN
7,6,NaN,NaN
8,7,NaN,NaN
9,8,CAVENDISH Mark,ASTANA
10,8,CRAS Steff,TOTALENERGIES


In [154]:
# cambiar el nombre de las columna etapas por stage_id

df4 = df4.rename(columns={'etapa': 'stage_id'})

df4

# cambiar el nombre de la columna correodr por rider_name

df4 = df4.rename(columns={'corredor': 'rider_name'})

df4

# cambiar el nombre de la columna equipo por team_name

df4 = df4.rename(columns={'equipo': 'team_name'})

df4



,stage_id,rider_name,team_name
1,1,MAS Enric,MOVISTAR
2,2,CARAPAZ Richard,EF
3,3,NaN,NaN
4,4,NaN,NaN
5,5,GUARNIERI Jacopo,LOTTO
6,5,SANCHEZ Luis Leon,NaN
7,6,NaN,NaN
8,7,NaN,NaN
9,8,CAVENDISH Mark,ASTANA
10,8,CRAS Steff,TOTALENERGIES


In [157]:
# reiniciar el índice del dataframe desde 0

df4 = df4.reset_index(drop=True)

df4

,stage_id,rider_name,team_name,withdrawal_id
0,1,MAS Enric,MOVISTAR,2
1,2,CARAPAZ Richard,EF,3
2,3,NaN,NaN,4
3,4,NaN,NaN,5
4,5,GUARNIERI Jacopo,LOTTO,6
5,5,SANCHEZ Luis Leon,NaN,7
6,6,NaN,NaN,8
7,7,NaN,NaN,9
8,8,CAVENDISH Mark,ASTANA,10
9,8,CRAS Steff,TOTALENERGIES,11


In [158]:
# crear una columna llamada withdrawal_id con el número de abandono al principio del dataframe

df4['withdrawal_id'] = df4.index + 1

df4

,stage_id,rider_name,team_name,withdrawal_id
0,1,MAS Enric,MOVISTAR,1
1,2,CARAPAZ Richard,EF,2
2,3,NaN,NaN,3
3,4,NaN,NaN,4
4,5,GUARNIERI Jacopo,LOTTO,5
5,5,SANCHEZ Luis Leon,NaN,6
6,6,NaN,NaN,7
7,7,NaN,NaN,8
8,8,CAVENDISH Mark,ASTANA,9
9,8,CRAS Steff,TOTALENERGIES,10


In [159]:
# poner la columna withdrawal_id al principio del dataframe

df4 = df4[['withdrawal_id', 'stage_id', 'rider_name', 'team_name']]

df4

,withdrawal_id,stage_id,rider_name,team_name
0,1,1,MAS Enric,MOVISTAR
1,2,2,CARAPAZ Richard,EF
2,3,3,NaN,NaN
3,4,4,NaN,NaN
4,5,5,GUARNIERI Jacopo,LOTTO
5,6,5,SANCHEZ Luis Leon,NaN
6,7,6,NaN,NaN
7,8,7,NaN,NaN
8,9,8,CAVENDISH Mark,ASTANA
9,10,8,CRAS Steff,TOTALENERGIES


In [169]:
# importar el csv withdrawals como dataframe

df5 = pd.read_csv('withdrawals.csv')

df5

# cambiar el nombre de las columna etapas por stage_id

df5 = df5.rename(columns={'etapa': 'stage_id'})

df5

# cambiar el nombre de la columna correodr por rider_name

df5 = df5.rename(columns={'corredor': 'rider_name'})

df5

# cambiar el nombre de la columna equipo por team_name

df5 = df5.rename(columns={'equipo': 'team_name'})

df5

# reiniciar el índice del dataframe desde 0

df5 = df5.reset_index(drop=True)

df5

# crear una columna llamada withdrawal_id con el número de abandono al principio del dataframe

df5['withdrawal_id'] = df5.index + 1

df5

# poner la columna withdrawal_id al principio del dataframe

df5 = df5[['withdrawal_id', 'stage_id', 'rider_name', 'team_name']]

df5



,withdrawal_id,stage_id,rider_name,team_name
0,1,1,MAS Enric,movistar_team
1,2,2,CARAPAZ Richard,ef_education_easypost
2,3,3,NaN,NaN
3,4,4,NaN,NaN
4,5,5,GUARNIERI Jacopo,lotto_dstny
5,6,5,SANCHEZ Luis Leon,astana_qazaqstan_team
6,7,6,NaN,NaN
7,8,7,NaN,NaN
8,9,8,CAVENDISH Mark,astana_qazaqstan_team
9,10,8,CRAS Steff,TOTALENERGIES


In [171]:
# cambiar los elementos de la columna rider_name a minuscúlas y cambiar los espacios por guiones bajos

df5['rider_name'] = df5['rider_name'].str.lower().str.replace(' ', '_')

df5

,withdrawal_id,stage_id,rider_name,team_name
0,1,1,mas_enric,movistar_team
1,2,2,carapaz_richard,ef_education_easypost
2,3,3,NaN,NaN
3,4,4,NaN,NaN
4,5,5,guarnieri_jacopo,lotto_dstny
5,6,5,sanchez_luis_leon,astana_qazaqstan_team
6,7,6,NaN,NaN
7,8,7,NaN,NaN
8,9,8,cavendish_mark,astana_qazaqstan_team
9,10,8,cras_steff,TOTALENERGIES


In [172]:
# cambiar los elemnetos de la columna team_name a minúsculas y cambiar los espacios por guiones bajos

df5['team_name'] = df5['team_name'].str.lower().str.replace(' ', '_')

df5

,withdrawal_id,stage_id,rider_name,team_name
0,1,1,mas_enric,movistar_team
1,2,2,carapaz_richard,ef_education_easypost
2,3,3,NaN,NaN
3,4,4,NaN,NaN
4,5,5,guarnieri_jacopo,lotto_dstny
5,6,5,sanchez_luis_leon,astana_qazaqstan_team
6,7,6,NaN,NaN
7,8,7,NaN,NaN
8,9,8,cavendish_mark,astana_qazaqstan_team
9,10,8,cras_steff,totalenergies


In [174]:
# cambiar los nan de la columna team_name por 'no withdrawal'

df5['team_name'] = df5['team_name'].fillna('no_withdrawal')

df5

# cambiar los nan de la columna rider_name por 'no withdrawal'

df5['rider_name'] = df5['rider_name'].fillna('no_withdrawal')

df5

,withdrawal_id,stage_id,rider_name,team_name
0,1,1,mas_enric,movistar_team
1,2,2,carapaz_richard,ef_education_easypost
2,3,3,no_withdrawal,no_withdrawal
3,4,4,no_withdrawal,no_withdrawal
4,5,5,guarnieri_jacopo,lotto_dstny
5,6,5,sanchez_luis_leon,astana_qazaqstan_team
6,7,6,no_withdrawal,no_withdrawal
7,8,7,no_withdrawal,no_withdrawal
8,9,8,cavendish_mark,astana_qazaqstan_team
9,10,8,cras_steff,totalenergies


In [175]:
# exportar el dataframe a un csv

df5.to_csv('withdrawals.csv')